# pandas函数应用——apply、applymap、map方法
这个网站有numpy、pandas、matplotlib等数据分析和机器学习的文章，不错的
https://finthon.com/pandas-apply-applymap-map/
## 简介
在pandas中， apply() 方法使用是非常灵活的，他比 agg() 方法使用更自由。数据分析师日常使用最多的就是 apply() 方法了，而与之类似的还有 applymap() 和 map() 方法，因此本文将详细介绍下这三种方法的使用和区别：

* apply：应用在DataFrame的行或列中；  
* applymap：应用在DataFrame的每个元素中；  
* map：应用在单独一列（Series）的每个元素中。  

知道它们的使用范围那就好办，接下来我们将会一一做介绍。

In [1]:
import numpy as np
import pandas as pd

## apply()方法
apply方法是一般性的“拆分-应用-合并”方法。它既可以得到一个经过广播的标量值，也可以得到一个相同大小的结果数组。我们先来看下函数形式：
```python
df.apply(func, axis=0, broadcast=None, raw=False, reduce=None, result_type=None, args=(), **kwds)
```
broadcast参数和reduce将在将来的版本中删除，替换为result_type ='broadcast'和result_type='reduce'。

In [3]:
df = pd.DataFrame(np.random.randint(10,size=(3,2)), columns=list('AB'))
df

,A,B
0,2,7
1,5,5
2,5,7


### apply基本使用

可以直接使用NumPy的函数：

In [5]:
#注意apply作用的是Series objects对象，不要被函数的返回结果给迷惑了，因为numpy和pandas都支持矢量化运算
#这个函数相当于在每一列的Series调用sqrt函数
df.apply(np.sqrt)

,A,B
0,1.414214,2.645751
1,2.236068,2.236068
2,2.236068,2.645751


In [6]:
# 默认为行运算
df.apply(np.sum)

A    12
B    19
dtype: int64

In [7]:
# axis=1列运算
df.apply(np.sum, axis=1)

0     9
1    10
2    12
dtype: int64

使用 lambda 函数做简单的运算：

In [8]:
df.apply(lambda x: x + 1)

,A,B
0,3,8
1,6,6
2,6,8


In [10]:
#看看apply函数的执行过程
def test(x):
    print(x)
    return x + 1
df.apply(test)

0    2
1    5
2    5
Name: A, dtype: int64
0    2
1    5
2    5
Name: A, dtype: int64
0    7
1    5
2    7
Name: B, dtype: int64


,A,B
0,3,8
1,6,6
2,6,8


卧槽，第一列调用了两次，怎么回事？看看官网的说明： 
In the current implementation apply calls func twice on the first column/row to decide whether it can take a fast or slow code path. This can lead to unexpected behavior if func has side-effects, as they will take effect twice for the first column/row.

百度翻译一下：在当前的实现中，对第一列/行应用两次调用func，以决定它是可以采用快速还是慢速的代码路径。如果func有副作用，这可能会导致意外的行为，因为它们将对第一列/第一行生效两次。

apply的使用是很灵活的，再举一个例子，配合 loc 方法我们能够在最后一行得到一个总和：

In [12]:
df.loc[3] = df.apply(np.sum)
df

,A,B
0,2,7
1,5,5
2,5,7
3,24,38


### apply自定义函数传递参数
在这里我们先定义了一个 cal_result 函数，它的作用是计算 A,B 列和的 x 倍和 y 倍添加到 C,D 列中。这里有三种方式可以完成参数的赋值，第一种方式直接通过关键字参数赋值，指定参数的值；第二种方式是使用 args 关键字参数传入一个包含参数的元组；第三种方式传入通过 ** 传入包含参数和值的字典。

In [13]:
#自定义函数
def cal_result(df, x, y):
    df['C'] = (df['A'] + df['B']) * x
    df['D'] = (df['A'] + df['B']) * y
    return df

In [14]:
df.apply(cal_result, x=3, y=8, axis=1)            # 第一种方式

,A,B,C,D
0,2,7,27,72
1,5,5,30,80
2,5,7,36,96
3,24,38,186,496


In [15]:
df.apply(cal_result, args=(3, 8), axis=1)         # 第二种方式

,A,B,C,D
0,2,7,27,72
1,5,5,30,80
2,5,7,36,96
3,24,38,186,496


In [16]:
df.apply(cal_result, **{'x': 3, 'y': 8}, axis=1)  # 第三种方式

,A,B,C,D
0,2,7,27,72
1,5,5,30,80
2,5,7,36,96
3,24,38,186,496


### expandt参数
直接引用官网的例子吧

In [17]:
df.apply(lambda x: [1, 2], axis=1)

0    [1, 2]
1    [1, 2]
2    [1, 2]
3    [1, 2]
dtype: object

In [18]:
#Passing result_type=’expand’ will expand list-like results to columns of a Dataframe
df.apply(lambda x: [1, 2], axis=1, result_type='expand')

,0,1
0,1,2
1,1,2
2,1,2
3,1,2


In [19]:
#Returning a Series inside the function is similar to passing result_type='expand'. 
#The resulting column names will be the Series index.
#返回Series就不需要expand参数了
df.apply(lambda x: pd.Series([1, 2], index=['foo', 'bar']), axis=1)

,foo,bar
0,1,2
1,1,2
2,1,2
3,1,2


### broadcast参数
Passing result_type='broadcast' will ensure the same shape result, whether list-like or scalar is returned by the function, and broadcast it along the axis. The resulting column names will be the originals.

传递结果“u type='broadcast”将确保相同的形状结果，无论函数返回的是类似列表的结果还是标量的结果，并沿轴进行广播。生成的列名称将是原始列名称。

In [21]:
df.apply(lambda x: [1,2], axis=1)

0    [1, 2]
1    [1, 2]
2    [1, 2]
3    [1, 2]
dtype: object

In [20]:
df.apply(lambda x: 1, axis=1, result_type='broadcast')

,A,B
0,1,1
1,1,1
2,1,1
3,1,1


In [22]:
df.apply(lambda x: [1,2], axis=1, result_type='broadcast')

,A,B
0,1,2
1,1,2
2,1,2
3,1,2


## applymap()方法
该方法针对DataFrame中的每个元素进行操作，还是使用这个数据：

In [24]:
df = pd.DataFrame(np.random.randint(10,size=(3,2)), columns=list('AB'))
df

,A,B
0,0,1
1,1,5
2,6,6


In [26]:
df.applymap(np.sqrt)

,A,B
0,0.00000,1.000000
1,1.00000,2.236068
2,2.44949,2.449490


In [27]:
df.applymap(lambda x : x + 1)

,A,B
0,1,2
1,2,6
2,7,7


In [28]:
df.applymap(lambda x : [x,1])

,A,B
0,"[0, 1]","[1, 1]"
1,"[1, 1]","[5, 1]"
2,"[6, 1]","[6, 1]"


## map()方法
map方法是应用在Series中的每个元素进行操作

In [30]:
df['A'].map(lambda x: x + 1)

0    1
1    2
2    7
Name: A, dtype: int64

In [29]:
df['A'].map(lambda x: '%.2f'%x)

0    0.00
1    1.00
2    6.00
Name: A, dtype: object